In [1]:
import torch
print(torch.__version__)

1.13.0


In [2]:
import torchtext
print(torchtext.__version__)

0.14.0


In [9]:
!pip install torchtext==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.0 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB ? eta 0:00:000:00:0100:02
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.0
    Uninstalling torchtext-0.14.0:
      Successfully uninstalled torchtext-0.14.0


In [1]:
import torch
print(torch.__version__)

1.8.0


In [2]:
import torchtext
print(torchtext.__version__)

0.9.0


In [3]:
# !python -m spacy download en

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenize(text):
    return [token.text for token in nlp(text)]

In [7]:
# Load the Penn Treebank POS-tagging dataset
dataset = torchtext.legacy.datasets.PennTreebank.splits(
    text_field = torchtext.legacy.data.Field(tokenize = tokenize),
)

In [9]:
train_dataset = dataset[0]
test_dataset = dataset[1]

In [34]:
train_dataset.

<generator object Dataset.__getattr__ at 0x7f4e064955f0>

In [35]:
# Build the vocabulary
vocab = torchtext.vocab.Vocab(train_dataset.text)

AttributeError: 'generator' object has no attribute 'copy'

In [21]:
# Define the fields for the dataset
text_field = torchtext.legacy.data.Field(tokenize = tokenize)
label_field = torchtext.legacy.data.Field(sequential = False)

In [27]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C dataset/

--2022-11-27 09:55:26--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  9.53MB/s    in 4.8s    

2022-11-27 09:55:32 (6.86 MB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [26]:
train_dataset, test_dataset = torchtext.legacy.data.TabularDataset.splits(
    path = "./dataset/",
    train = "ptb.train.txt",
    test = "ptb.test.txt",
    format = "tsv",
    fields = [("text", text_field), ("label", label_field)],
)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/dataset/ptb.train.txt'

In [37]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 8.7 MB/s eta 0:00:0000:01


In [38]:
import nltk
#uncomment if treebank not found
nltk.download('treebank')
from nltk.corpus import treebank

[nltk_data] Downloading package treebank to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [39]:
# Load training data
train_data = treebank.tagged_sents()

In [40]:
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [52]:
from collections import defaultdict
# Extract unique POS tags from training data
all_tags = [tag for example in train_data for _, tag in example]
tag_to_ix = defaultdict(lambda: len(tag_to_ix))
for tag in all_tags:
    tag_to_ix[tag]
# Extract unique lemmas from training data
all_wrd = [word for example in train_data for word, _ in example]
word_to_ix = defaultdict(lambda: len(word_to_ix))
for wrd in all_wrd:
    word_to_ix[wrd]

In [53]:
all_tags[:5], all_wrd[:5]

(['NNP', 'NNP', ',', 'CD', 'NNS'], ['Pierre', 'Vinken', ',', '61', 'years'])

In [54]:
# Create POS tag and lemma lookup dictionaries
tag_to_ix = {}
ix_to_tag = {}
word_to_ix = {}
ix_to_word = {}
for sentence in train_data:
    for word, tag in sentence:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
            ix_to_tag[len(ix_to_tag)] = tag
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            ix_to_word[len(ix_to_word)] = word
# Convert the training data to a list of feature/target pairs
train_dataset = [([tag_to_ix[tag] for _, tag in sentence],
               [word_to_ix[word] for word, _ in sentence])
              for sentence in train_data]

In [56]:
train_dataset[0]

([0, 0, 1, 2, 3, 4, 1, 5, 6, 7, 8, 9, 7, 4, 8, 0, 2, 10],
 [0, 1, 2, 3, 4, 5, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])

In [43]:
# Define the model
class POS_Tagger(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, text):
        # Shape: (num_words, batch_size)
        embedded = self.embedding(text)
        # Shape: (num_words, batch_size, embedding_dim)
        output, _ = self.rnn(embedded)
        # Shape: (num_words, batch_size, vocab_size)
        output = self.linear(output)
        return output

In [57]:
# Instantiate the model and define the loss and optimizer
model = POS_Tagger(len(all_wrd), 100, 100)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [58]:
model

POS_Tagger(
  (embedding): Embedding(100676, 100)
  (rnn): LSTM(100, 100)
  (linear): Linear(in_features=100, out_features=100676, bias=True)
)

In [45]:

# Train the model
for epoch in range(10):
    for text, tags in train_dataset:
        # Shape: (num_words, batch_size, vocab_size)
        output = model(text)
        # Shape: (num_words * batch_size, vocab_size)
        output = output.view(-1, output.shape[-1])
        # Shape: (num_words * batch_size,)
        tags = tags.view(-1)
        # Compute the loss
        loss = loss_fn(output, tags)
        # Perform backpropagation
        loss.backward()
        # Update the model parameters
        optimizer.step()

TypeError: cannot unpack non-iterable Example object

In [46]:
# Evaluate the model
correct = 0
total = 0

for text, tags in test_dataset:
    output = model(text)
    # Shape: (num_words, batch_size, vocab_size)
    _, predicted_tags = output.max(dim = -1)
    # Shape: (num_words, batch_size)
    correct += (predicted_tags == tags).sum().item()
    total += tags.numel()

accuracy = correct / total
print(f"Test accuracy: {accuracy:.4f}")

TypeError: cannot unpack non-iterable Example object

In [59]:
from itertools import chain
from collections import Counter
import operator

In [66]:
def get_data(filename):
    """Fungsi untuk melakukan load data pada file .tsv maupun .txt
    
    File yang digunakan yaitu file .tsv maupun file yang mampu dibuka
    menggunakan teks editor. Dalam file tersebut terdapat tag pembuka 
    dan diakhiri tag penutup  yang berfungsi sebagai penanda bahwa
    entitas tersebut termasuk ke dalam satu kalimat.
    
    Args:
        filename: string dari nama file yang akan diload datanya.
        
    Return:
        list dari kata-kata dan tags dengan index yang menunjukkan posisi kalimat tersebut.
    """
    
    # Load data dan buka sebagai file
    sentences = []
    tags = []
    with open(filename) as file:
        contents = file.readlines()
        
    print(contents)

    # Hapus karakter \n yang tidak dibutuhkan
    contents = [content.strip() for content in contents]
    idx = 0
    while idx < len(contents):
        word = []
        tag = []
        # looping sampai menemukan pattern dengan awalan 
        while not contents[idx].startswith("'"):
            # kondisi jika menemukan sebuah data yang tidak memiliki awalan 
            if not contents[idx].startswith("'"):
                temp_word, temp_tag = contents[idx].split("\t")
                word.append(temp_word.lower())
                tag.append(temp_tag)
            idx += 1
        sentences.append(word)
        tags.append(tag)
        idx += 2
        
    return sentences, tags

In [67]:
def flatten(multi_list):
    """Mengembalikan list multi dimensi ke dalam list satu dimensi.
    
    Input list yang masuk di proses menggunakan method from_iterable dari
    package itertools dan mengembalikan object berupa generator yang iterable.
    
    Args:
        multi_list: list multi dimensi.
    
    Return:
        list satu dimensi yang nantinya digunakan untuk pemrosesan lebih lanjut.
    """
    
    return chain.from_iterable(multi_list)

In [68]:
def get_emission_table(words, tags):
    """Membuat tabel emisi dari input kata dan tags.
    
    Fungsi untuk mengembalikan sebuah tabel dari kata dan kemunculan tags terhadap kata tersebut.
    
    Args:
        words: list kata.
        tags: list tags.
    
    Return:
        dictionary yang merepresntasikan tabel emisi.
    """
    
    # Membuat representasi tabel emission probability dari HMM
    hidden_state = {}
    for word in words:
        # Jika word belum pernah ditemui maka akan digenerate
        # tagset berserta probabilitasnya
        if word not in hidden_state.keys():
            word_tags = []
            for idx, wrd in enumerate(words):
                if wrd == word :
                    word_tags.append(tags[idx])
            # Membuat dictionary berisi semua tagset dari sebuah word
            # pada korpus. Berserta jumlah kemunculan setiap tagsetnya
            tag_count = {}
            for tag in word_tags:
                if tag not in tag_count.keys():
                    tag_count[tag] = 1
                else:
                    tag_count[tag] += 1
            total = 0
            # Mengubah jumlah kemunculan menjadi probabilitas
            tag_prob = {}
            for count in tag_count.values():
                total += count
            for tagset in tag_count.keys():
                tag_prob[tagset] = tag_count[tagset]/total
            hidden_state[word] = tag_prob
        else:
            # Jika word sudah pernah ditemui maka akan dilewati
            continue
    return(hidden_state)

In [69]:
filename = './dataset/corpus.tsv'
sentences, tags = get_data(filename)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ValueError: not enough values to unpack (expected 2, got 1)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List, Callable, Dict
from collections import OrderedDict
from itertools import groupby
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

ModuleNotFoundError: No module named 'tensorflow'